### Task 1:

Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/ 

In [10]:
import pandas as pd
from pandasql import sqldf
import sqlite3
from sqlalchemy import create_engine
import functools as func

In [11]:
dfAdult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)

In [12]:
dfAdult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
dfColumnNames = pd.read_fwf('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names', sep="\n")
dfColumnNames.columns

Index(['| This data was extracted from the census bureau database found at',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')

In [14]:
dfColumnNames['RequiredData'] = dfColumnNames['| This data was extracted from the census bureau database found at']

In [15]:
dfColumnNames = dfColumnNames.drop(columns={'| This data was extracted from the census bureau database found at',
       'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'})
columnNames = dfColumnNames.iloc[95:,]['RequiredData'].str.split(":")
columnNames = [i[0] for i in columnNames]

In [16]:
#Last column name is missing so add it manually
columnNames.append('salary')

In [17]:
columnNames = func.reduce(lambda x,y : x + ',' + y, columnNames)
columnNames

'age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary'

In [18]:
columnNames = list(columnNames.replace('-', '_').split(','))

In [19]:
dfAdult.columns = columnNames

In [20]:
dfAdult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [21]:
#engine = create_engine('sqlite:///testDB', echo=True)
conn = sqlite3.connect('testDB')
cursor = conn.cursor()

In [22]:

import functools as func
columnNames = func.reduce(lambda x,y : x + ' %s ,' + y, dfAdult.columns.values) + ' %s'
columnNames

'age %s ,workclass %s ,fnlwgt %s ,education %s ,education_num %s ,marital_status %s ,occupation %s ,relationship %s ,race %s ,sex %s ,capital_gain %s ,capital_loss %s ,hours_per_week %s ,native_country %s ,salary %s'

In [23]:
columnNames2 = columnNames %('INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'INTEGER', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'INTEGER', 'INTEGER', 'INTEGER', 'TEXT', 'TEXT')
columnNames2

'age INTEGER ,workclass TEXT ,fnlwgt INTEGER ,education TEXT ,education_num INTEGER ,marital_status TEXT ,occupation TEXT ,relationship TEXT ,race TEXT ,sex TEXT ,capital_gain INTEGER ,capital_loss INTEGER ,hours_per_week INTEGER ,native_country TEXT ,salary TEXT'

In [24]:
'create table sqldb3(%s)' %columnNames2

'create table sqldb3(age INTEGER ,workclass TEXT ,fnlwgt INTEGER ,education TEXT ,education_num INTEGER ,marital_status TEXT ,occupation TEXT ,relationship TEXT ,race TEXT ,sex TEXT ,capital_gain INTEGER ,capital_loss INTEGER ,hours_per_week INTEGER ,native_country TEXT ,salary TEXT)'

In [25]:
cursor.execute('create table sqldb3(%s)' %columnNames2)

In [26]:
sqlInsert = """insert into sqldb3 values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""

In [27]:
columnNames3 = func.reduce(lambda x,y: x + ', ' + 'dfAdult.' + y, dfAdult.columns.values)
columnNames3 = 'dfAdult.' + columnNames3
columnNames3

'dfAdult.age, dfAdult.workclass, dfAdult.fnlwgt, dfAdult.education, dfAdult.education_num, dfAdult.marital_status, dfAdult.occupation, dfAdult.relationship, dfAdult.race, dfAdult.sex, dfAdult.capital_gain, dfAdult.capital_loss, dfAdult.hours_per_week, dfAdult.native_country, dfAdult.salary'

In [28]:

cursor.executemany(sqlInsert, zip(dfAdult.age, dfAdult.workclass, dfAdult.fnlwgt, dfAdult.education, dfAdult.education_num, dfAdult.marital_status, dfAdult.occupation, dfAdult.relationship, dfAdult.race, dfAdult.sex, dfAdult.capital_gain, dfAdult.capital_loss, dfAdult.hours_per_week, dfAdult.native_country, dfAdult.salary))

In [29]:
conn.commit()

## 1. Select 10 records from the adult sqladb

In [30]:
cursor.execute('select * from sqldb3 limit 10;')
cursor.fetchall()

[(39,
  ' State-gov',
  77516,
  ' Bachelors',
  13,
  ' Never-married',
  ' Adm-clerical',
  ' Not-in-family',
  ' White',
  ' Male',
  2174,
  0,
  40,
  ' United-States',
  ' <=50K'),
 (50,
  ' Self-emp-not-inc',
  83311,
  ' Bachelors',
  13,
  ' Married-civ-spouse',
  ' Exec-managerial',
  ' Husband',
  ' White',
  ' Male',
  0,
  0,
  13,
  ' United-States',
  ' <=50K'),
 (38,
  ' Private',
  215646,
  ' HS-grad',
  9,
  ' Divorced',
  ' Handlers-cleaners',
  ' Not-in-family',
  ' White',
  ' Male',
  0,
  0,
  40,
  ' United-States',
  ' <=50K'),
 (53,
  ' Private',
  234721,
  ' 11th',
  7,
  ' Married-civ-spouse',
  ' Handlers-cleaners',
  ' Husband',
  ' Black',
  ' Male',
  0,
  0,
  40,
  ' United-States',
  ' <=50K'),
 (28,
  ' Private',
  338409,
  ' Bachelors',
  13,
  ' Married-civ-spouse',
  ' Prof-specialty',
  ' Wife',
  ' Black',
  ' Female',
  0,
  0,
  40,
  ' Cuba',
  ' <=50K'),
 (37,
  ' Private',
  284582,
  ' Masters',
  14,
  ' Married-civ-spouse',
  ' Exec-m

## 2. Show me the average hours per week of all men who are working in private sector

In [31]:
cursor.execute('select avg(hours_per_week) from sqldb3 where workclass=" Private";')
cursor.fetchall()

[(40.267095523440254,)]

## 3. Show me the frequency table for education, occupation and relationship, separately

In [32]:
cursor.execute('select count(education) from sqldb3;')
print('education: ')
print(cursor.fetchall())
cursor.execute('select count(occupation) from sqldb3;')
print('occupation: ')
print(cursor.fetchall())
cursor.execute('select count(relationship) from sqldb3;')
print('relationship: ')
print(cursor.fetchall())

education: 
[(32561,)]
occupation: 
[(32561,)]
relationship: 
[(32561,)]


In [33]:
cursor.execute('select education, count(education), count(occupation), count(relationship) from sqldb3 group by education;')
print('education, total#education, total#occupation, total#relationship')
print('-'*70)
returnedRows = cursor.fetchall()
[print(x) for x in returnedRows]

education, total#education, total#occupation, total#relationship
----------------------------------------------------------------------
(' 10th', 933, 933, 933)
(' 11th', 1175, 1175, 1175)
(' 12th', 433, 433, 433)
(' 1st-4th', 168, 168, 168)
(' 5th-6th', 333, 333, 333)
(' 7th-8th', 646, 646, 646)
(' 9th', 514, 514, 514)
(' Assoc-acdm', 1067, 1067, 1067)
(' Assoc-voc', 1382, 1382, 1382)
(' Bachelors', 5355, 5355, 5355)
(' Doctorate', 413, 413, 413)
(' HS-grad', 10501, 10501, 10501)
(' Masters', 1723, 1723, 1723)
(' Preschool', 51, 51, 51)
(' Prof-school', 576, 576, 576)
(' Some-college', 7291, 7291, 7291)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## 4. Are there any people who are married, working in private sector and having a masters degree

In [34]:
dfAdult.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [35]:
cursor.execute("""select count(*) from sqldb3 where marital_status like "%Married-civ-spouse"
and workclass like "%Private" and education like "%Masters";""")
cursor.fetchall()

[(531,)]

## 5. What is the average, minimum and maximum age group for people working in different sectors

In [36]:
cursor.execute("""select workclass, min(age), avg(age), max(age) from sqldb3 group by workclass;""")
cursor.fetchall()

[(' ?', 17, 40.960239651416124, 90),
 (' Federal-gov', 17, 42.590625, 90),
 (' Local-gov', 17, 41.75107501194458, 90),
 (' Never-worked', 17, 20.571428571428573, 30),
 (' Private', 17, 36.797585477617204, 90),
 (' Self-emp-inc', 17, 46.01702508960574, 84),
 (' Self-emp-not-inc', 17, 44.96969696969697, 90),
 (' State-gov', 17, 39.43605546995377, 81),
 (' Without-pay', 19, 47.785714285714285, 72)]

## 6. Calculate age distribution by country

In [37]:
cursor.execute("""select native_country, min(age), avg(age), max(age) from sqldb3 
group by native_country;""")
print('Country, minimum age, average age, maximum age')
print('-'*70)
cursor.fetchall()

Country, minimum age, average age, maximum age
----------------------------------------------------------------------


[(' ?', 17, 38.72555746140652, 90),
 (' Cambodia', 18, 37.78947368421053, 65),
 (' Canada', 17, 42.54545454545455, 80),
 (' China', 22, 42.53333333333333, 75),
 (' Columbia', 18, 39.71186440677966, 75),
 (' Cuba', 21, 45.76842105263158, 82),
 (' Dominican-Republic', 18, 37.72857142857143, 78),
 (' Ecuador', 21, 36.642857142857146, 90),
 (' El-Salvador', 17, 34.132075471698116, 79),
 (' England', 17, 41.15555555555556, 90),
 (' France', 20, 38.96551724137931, 64),
 (' Germany', 18, 39.25547445255474, 74),
 (' Greece', 22, 46.206896551724135, 65),
 (' Guatemala', 19, 32.421875, 66),
 (' Haiti', 17, 38.27272727272727, 63),
 (' Holand-Netherlands', 32, 32.0, 32),
 (' Honduras', 18, 33.84615384615385, 58),
 (' Hong', 19, 33.65, 60),
 (' Hungary', 24, 49.38461538461539, 81),
 (' India', 17, 38.09, 61),
 (' Iran', 22, 39.41860465116279, 63),
 (' Ireland', 23, 36.458333333333336, 68),
 (' Italy', 19, 46.42465753424658, 77),
 (' Jamaica', 18, 35.592592592592595, 66),
 (' Japan', 19, 38.24193548

## 7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

In [38]:
cursor.execute("""select (capital_gain - capital_loss) as Net_Capital_Gain from sqldb3
where capital_gain != 0;""")
print('Net Capital Gain for 2 rows')
print(cursor.fetchone())
print(cursor.fetchone())

Net Capital Gain for 2 rows
(2174,)
(14084,)


In [39]:
cursor.execute("""select native_country as Country, sum(capital_gain - capital_loss) as Net_Capital_Gain from sqldb3
where capital_gain != 0 group by native_country;""")
print('Net Capital Gain for every country')
cursor.fetchall()

Net Capital Gain for every country


[(' ?', 1053200),
 (' Cambodia', 19529),
 (' Canada', 182000),
 (' China', 27486),
 (' Columbia', 10656),
 (' Cuba', 34887),
 (' Dominican-Republic', 103324),
 (' Ecuador', 14773),
 (' El-Salvador', 60878),
 (' England', 77825),
 (' France', 15864),
 (' Germany', 121532),
 (' Greece', 37233),
 (' Guatemala', 14773),
 (' Hungary', 9316),
 (' India', 360492),
 (' Iran', 87920),
 (' Ireland', 7787),
 (' Italy', 44640),
 (' Jamaica', 38486),
 (' Japan', 157866),
 (' Laos', 2885),
 (' Mexico', 223139),
 (' Nicaragua', 6794),
 (' Peru', 1831),
 (' Philippines', 201632),
 (' Poland', 41057),
 (' Portugal', 8450),
 (' Puerto-Rico', 36160),
 (' Scotland', 5178),
 (' South', 149579),
 (' Taiwan', 109499),
 (' United-States', 31772837),
 (' Vietnam', 44260),
 (' Yugoslavia', 5556)]

# Task 2:

### Read the following data set: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

In [41]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, MetaData, Table, select

In [42]:
dfAdultData = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)

In [43]:
columnNamesTask2 = columnNames.replace("%s", "")
columnNamesTask22 = columnNamesTask2.split("  ,")
columnNamesTask22

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'salary ']

In [44]:
dfAdultData.columns = columnNamesTask22

In [45]:
dfAdultData.head(3)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


## 1. Create an sqlalchemy engine using a sample from the data set

In [46]:
engine = create_engine('sqlite:///adultData.db', echo=True)

In [47]:
dfAdultData.to_sql('Adult', engine)

2019-07-20 21:51:50,197 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-20 21:51:50,199 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:51:50,201 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-20 21:51:50,201 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:51:50,203 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult")
2019-07-20 21:51:50,204 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:51:50,211 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Adult" (
	"index" BIGINT, 
	age BIGINT, 
	workclass TEXT, 
	fnlwgt BIGINT, 
	education TEXT, 
	education_num BIGINT, 
	marital_status TEXT, 
	occupation TEXT, 
	relationship TEXT, 
	race TEXT, 
	sex TEXT, 
	capital_gain BIGINT, 
	capital_loss BIGINT, 
	hours_per_week BIGINT, 
	native_country TEXT, 
	"salary " TEXT
)


2019-07-20 21:51:50,213 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:51:50,226 INFO sqlal

## 2. Write two basic update queries

In [48]:
metadata = MetaData()

In [49]:
user = Table('Adult', metadata,
    Column('index', Integer, primary_key=True),
    Column('age', Integer),
    Column('workclass', String),
    Column('nlwgt', Integer),
    Column('education', String),
    Column('education_num', Integer),
    Column('marital_status', String),
    Column('occupation', String),
    Column('relationship', String),
    Column('race', String),
    Column('sex', String),
    Column('capital_gain', Integer),
    Column('capital_loss', Integer),
    Column('hours_per_week', Integer),
    Column('native_country', String),
    Column('salary', String),
    extend_existing=True,
    autoload=True,
    autoload_with=engine
)

2019-07-20 21:52:03,554 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Adult")
2019-07-20 21:52:03,556 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:52:03,560 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Adult' AND type = 'table'
2019-07-20 21:52:03,561 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:52:03,565 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Adult")
2019-07-20 21:52:03,566 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:52:03,567 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Adult' AND type = 'table'
2019-07-20 21:52:03,568 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:52:03,572 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("Adult")
2019-07-20 21:52:03,573 INFO sqlalchemy.engine.base.Engine ()
2019-07-20 21:52:03,575 INFO sqlalchemy.engi

In [50]:
upd = user.update().where(user.c.native_country==' Cambodia').values(native_country='Cambodia')

In [51]:
engine.execute(upd)

2019-07-20 21:52:12,336 INFO sqlalchemy.engine.base.Engine UPDATE "Adult" SET native_country=? WHERE "Adult".native_country = ?
2019-07-20 21:52:12,338 INFO sqlalchemy.engine.base.Engine ('Cambodia', ' Cambodia')
2019-07-20 21:52:12,367 INFO sqlalchemy.engine.base.Engine COMMIT


In [52]:
upd = user.update().where(user.c.fnlwgt==1455435).values(fnlwgt=1455435)

In [53]:
engine.execute(upd)

2019-07-20 21:52:24,943 INFO sqlalchemy.engine.base.Engine UPDATE "Adult" SET fnlwgt=? WHERE "Adult".fnlwgt = ?
2019-07-20 21:52:24,945 INFO sqlalchemy.engine.base.Engine (1455435, 1455435)
2019-07-20 21:52:24,964 INFO sqlalchemy.engine.base.Engine COMMIT


In [54]:
# One more way of update query
import sqlite3
connection = sqlite3.connect("adultData.db")
cursor = connection.cursor()
cursor.execute('update Adult set native_country="Cambodia" where native_country = "%Cambodia"')
cursor.execute('update Adult set education=trim(education)')

## 3. Write two delete queries

In [ ]:
dele = user.delete().where(user.c.capital_gain==99999)
engine.execute(dele)

In [ ]:
dele = user.delete().where(user.c.fnlwgt==1455435)
engine.execute(dele)

In [59]:

# One more way of delete query
cursor.execute('select distinct(workclass) from Adult where capital_gain=99999').fetchone()
cursor.execute('delete from Adult where capital_gain=99999 and workclass like "%Self-emp-inc"')
cursor.execute('select max(fnlwgt) from Adult').fetchone()
cursor.execute('delete from Adult where fnlwgt=1455435')

## 4. Write two filter queries

In [60]:
# As explained in support ticket, Table object doesn't has filter_by to query with filters and Base class object doesn't have extend_existing so I am 
# using SQL method to fetch rows by where clause

In [61]:
cursor.execute('select education as "Education", education_num as "Education_No" from Adult where education_num > 10').fetchone()

('Bachelors', 13)

In [62]:
cursor.execute('select avg(age) as "Age", native_country as "Country" from Adult where age > 40').fetchone()

(52.02095138681778, ' United-States')

## 5. Write two function queries

In [63]:
cursor.execute('select trim(marital_status), marital_status from Adult').fetchone()

('Never-married', ' Never-married')

In [64]:
cursor.execute('select avg(age), max(age) from Adult').fetchone()

(38.56841101955479, 90)